In [1]:
import numpy as np
import pandas as pd
import xarray as xr


## Collect predictions; format and save coSIF datasets

In [2]:
months = {"202102": "2021-02", "202104": "2021-04", "202107": "2021-07", "202110": "2021-10"}

for label, month in months.items():
    if label == "202107":
        method = "cokriging"
    else:
        method = "kriging"

    ds = xr.open_dataset(f"../data/intermediate/{method}_results_{label}.nc4").drop_vars("time")
    ds._attrs = {}

    ds = (
        ds[["predictions", "rmspe"]]
        .rename({"predictions": "cosif_prediction", "rmspe": "cosif_rmspe"})
        .assign_coords(month=pd.to_datetime(month))
        .assign_attrs(units="W/m^2/sr/μm")
        .to_netcdf(f"../data/output/coSIF_{label}.nc4", format="NETCDF4")
    )


## Calculate average standard error ratios (compared to the data)

In [5]:
format_month = lambda ds: ds.expand_dims(month=[pd.to_datetime(ds["month"].data)], axis=-1)
ds_cosif = xr.open_mfdataset("../data/output/coSIF_*.nc4", preprocess=format_month)
ds_cosif


<xarray.Dataset>
Dimensions:           (month: 4, lat: 720, lon: 1200)
Coordinates:
  * month             (month) datetime64[ns] 2021-02-01 ... 2021-10-01
  * lat               (lat) float64 22.02 22.07 22.12 ... 57.87 57.92 57.97
  * lon               (lon) float64 -125.0 -124.9 -124.9 ... -65.07 -65.02
Data variables:
    cosif_prediction  (lat, lon, month) float64 dask.array<chunksize=(720, 1200, 1), meta=np.ndarray>
    cosif_rmspe       (lat, lon, month) float64 dask.array<chunksize=(720, 1200, 1), meta=np.ndarray>
Attributes:
    units:    W/m^2/sr/μm

In [6]:
with xr.open_dataset(
    "../data/intermediate/OCO2_005deg_months2021_north_america_with_basis.nc4"
) as ds:
    ds_data = ds.rename(time="month").sel(month=ds_cosif.month)

ds_data


<xarray.Dataset>
Dimensions:   (lat: 720, lon: 1200, month: 4)
Coordinates:
  * lat       (lat) float64 22.02 22.07 22.12 22.17 ... 57.82 57.87 57.92 57.97
  * lon       (lon) float64 -125.0 -124.9 -124.9 -124.8 ... -65.12 -65.07 -65.02
  * month     (month) datetime64[ns] 2021-02-01 2021-04-01 2021-07-01 2021-10-01
Data variables: (12/64)
    sif       (lat, lon, month) float32 ...
    sif_var   (lat, lon, month) float32 ...
    xco2      (lat, lon, month) float32 ...
    xco2_var  (lat, lon, month) float32 ...
    B1        (lat, lon) float64 ...
    B2        (lat, lon) float64 ...
    ...        ...
    B55       (lat, lon) float64 ...
    B56       (lat, lon) float64 ...
    B57       (lat, lon) float64 ...
    B58       (lat, lon) float64 ...
    B59       (lat, lon) float64 ...
    B60       (lat, lon) float64 ...

In [7]:
df_se = (
    xr.merge([ds_cosif["cosif_rmspe"], ds_data["sif_var"]])
    .squeeze()
    .astype(np.float64)
    .to_dataframe()
    .reset_index()
    .dropna(subset="sif_var")
)
df_se["me_sd"] = np.sqrt(df_se["sif_var"])  # measurement error standard deviation
df_se["se_ratio"] = df_se["cosif_rmspe"] / df_se["me_sd"]
error_ratio_means = df_se.groupby("month")["se_ratio"].mean()
error_ratio_means


month
2021-02-01    0.287380
2021-04-01    0.267288
2021-07-01    0.253762
2021-10-01    0.253446
Name: se_ratio, dtype: float64